In [1]:
import sys
import platform
import torch
import tensorflow as tf

def get_platform_info():
    print(f"--- 💻 Системная Информация ---")
    print(f"OS: {platform.system()} {platform.release()} ({platform.machine()})")
    print(f"Python: {sys.version.split()[0]}")
    print("-" * 30 + "\n")

def check_pytorch():
    print("--- 🔥 PyTorch Диагностика ---")
    print(f"Version: {torch.__version__}")
    
    # CUDA Check
    cuda_available = torch.cuda.is_available()
    print(f"CUDA Available: {cuda_available}")
    if cuda_available:
        print(f"  - Device: {torch.cuda.get_device_name(0)}")
        print(f"  - CUDA Version: {torch.version.cuda}")
        print(f"  - cuDNN: {torch.backends.cudnn.version()}")
    
    # MPS Check (Apple Silicon)
    mps_available = False
    if platform.system() == "Darwin":
        # Проверка атрибута для обратной совместимости со старыми версиями torch
        mps_available = getattr(torch.backends, "mps", None) is not None and torch.backends.mps.is_available()
        print(f"MPS (Apple Silicon) Available: {mps_available}")
    
    # Тестовое вычисление
    device_type = "cuda" if cuda_available else ("mps" if mps_available else "cpu")
    device = torch.device(device_type)
    
    try:
        x = torch.randn(2000, 2000, device=device)
        _ = torch.mm(x, x.t())
        print(f"✅ Вычисления на [{device_type.upper()}] успешны.")
    except Exception as e:
        print(f"❌ Ошибка вычислений на {device_type}: {e}")
    print()

def check_tensorflow():
    print("--- ❄️ TensorFlow Диагностика ---")
    print(f"Version: {tf.__version__}")
    
    # Общий список GPU
    physical_devices = tf.config.list_physical_devices('GPU')
    print(f"Physical GPUs: {len(physical_devices)}")
    for i, dev in enumerate(physical_devices):
        print(f"  - GPU {i}: {dev.name} ({dev.device_type})")
    
    # Проверка специфики Metal (для macOS)
    if platform.system() == "Darwin":
        # В TF 2.x для Mac используется плагин tensorflow-metal
        is_metal = any("GPU" in dev.device_type for dev in physical_devices)
        print(f"Metal Acceleration: {'Active' if is_metal else 'Not Found'}")

    # Тестовое вычисление
    try:
        with tf.device('/GPU:0' if physical_devices else '/CPU:0'):
            a = tf.random.normal([2000, 2000])
            b = tf.matmul(a, tf.transpose(a))
            print(f"✅ Вычисления на {'GPU' if physical_devices else 'CPU'} успешны.")
    except Exception as e:
        print(f"❌ Ошибка вычислений TF: {e}")

if __name__ == "__main__":
    get_platform_info()
    check_pytorch()
    check_tensorflow()

--- 💻 Системная Информация ---
OS: Darwin 25.2.0 (arm64)
Python: 3.11.9
------------------------------

--- 🔥 PyTorch Диагностика ---
Version: 2.9.1
CUDA Available: False
MPS (Apple Silicon) Available: True
✅ Вычисления на [MPS] успешны.

--- ❄️ TensorFlow Диагностика ---
Version: 2.15.0
Physical GPUs: 1
  - GPU 0: /physical_device:GPU:0 (GPU)
Metal Acceleration: Active
✅ Вычисления на GPU успешны.


2025-12-27 01:39:13.798595: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Pro
2025-12-27 01:39:13.798609: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 48.00 GB
2025-12-27 01:39:13.798611: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 18.72 GB
2025-12-27 01:39:13.798651: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-12-27 01:39:13.798679: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [2]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

Using device: mps
